<a href="https://colab.research.google.com/github/DSE-Group7/5g-energy-consumption-modeling/blob/main/Notebooks/Kajaani/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc
import os
import itertools
import pickle
import re
import time

import warnings
warnings.filterwarnings('ignore')

from random import choice, choices
from functools import reduce
from tqdm import tqdm
from itertools import cycle

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
%matplotlib inline

from functools import reduce
from tqdm import tqdm
from itertools import cycle
from scipy import stats
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble
from sklearn import decomposition
from sklearn import tree

import lightgbm as lgb
import xgboost as xgb

pd.set_option("display.max_columns", None)

plt.style.use("ggplot")
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cell = pd.read_csv("/content/drive/MyDrive/competitions/input/CLdata.csv")
base = pd.read_csv("/content/drive/MyDrive/competitions/input/BSinfo.csv")
energy = pd.read_csv("/content/drive/MyDrive/competitions/input/ECdata.csv")

In [ ]:
base.head()

,BS,CellName,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,B_0,Cell0,Type1,Mode2,365.0,20,4,6.875934
1,B_1,Cell0,Type2,Mode2,532.0,20,4,6.875934
2,B_2,Cell0,Type1,Mode2,365.0,20,4,6.875934
3,B_3,Cell0,Type2,Mode2,532.0,20,4,6.875934
4,B_4,Cell0,Type2,Mode2,532.0,20,4,6.875934


In [ ]:
cell.head()

,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6
0,1/1/2023 1:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0
1,1/1/2023 2:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0
2,1/1/2023 3:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0
3,1/1/2023 4:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0
4,1/1/2023 5:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0


In [ ]:
energy.head()

,Time,BS,Energy
0,1/1/2023 1:00,B_0,64.275037
1,1/1/2023 2:00,B_0,55.904335
2,1/1/2023 3:00,B_0,57.698057
3,1/1/2023 4:00,B_0,55.156951
4,1/1/2023 5:00,B_0,56.053812


In [ ]:
# merge base and cell
base_cell = cell.merge(base, on=['BS','CellName'], how='left')
base_cell.head()

,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,1/1/2023 1:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
1,1/1/2023 2:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
2,1/1/2023 3:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
3,1/1/2023 4:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
4,1/1/2023 5:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934


In [ ]:
data = energy.merge(base_cell, on=['Time','BS'], how='left')
data.head(10)

,Time,BS,Energy,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,1/1/2023 1:00,B_0,64.275037,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
1,1/1/2023 2:00,B_0,55.904335,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
2,1/1/2023 3:00,B_0,57.698057,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
3,1/1/2023 4:00,B_0,55.156951,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
4,1/1/2023 5:00,B_0,56.053812,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
5,1/1/2023 7:00,B_0,82.959641,Cell0,0.351585,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
6,1/1/2023 8:00,B_0,91.031390,Cell0,0.759298,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
7,1/1/2023 9:00,B_0,78.176383,Cell0,0.887798,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
8,1/1/2023 10:00,B_0,72.645740,Cell0,0.624745,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
9,1/1/2023 14:00,B_0,66.666667,Cell0,0.437926,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934


In [ ]:
data.shape

(98084, 17)

In [ ]:
data.isnull().sum()

Time         0
BS           0
Energy       0
CellName     0
load         0
ESMode1      0
ESMode2      0
ESMode3      0
ESMode4      0
ESMode5      0
ESMode6      0
RUType       0
Mode         0
Frequency    0
Bandwidth    0
Antennas     0
TXpower      0
dtype: int64

In [ ]:
# drop Time, ESMode3,ESMode4,ESMode5
data.drop(['ESMode3','ESMode4','ESMode5'], axis=1, inplace=True)
data.head()

,Time,BS,Energy,CellName,load,ESMode1,ESMode2,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,1/1/2023 1:00,B_0,64.275037,Cell0,0.487936,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
1,1/1/2023 2:00,B_0,55.904335,Cell0,0.344468,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
2,1/1/2023 3:00,B_0,57.698057,Cell0,0.193766,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
3,1/1/2023 4:00,B_0,55.156951,Cell0,0.222383,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
4,1/1/2023 5:00,B_0,56.053812,Cell0,0.175436,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934


In [ ]:
# df = data.pivot(
#     index=['Time', 'BS'],
#     columns=['CellName'],
#     values=['load', 'ESMode1', 'ESMode2', 'ESMode6', 'Frequency',
#        'Bandwidth', 'Antennas', 'TXpower'],
# ).reset_index()

# df.columns = ['_'.join([str(i) for i in x]) for x in df.columns]
# df.columns = df.columns.str.strip('_')
# df = df.merge(base.groupby('BS')[['RUType', 'Mode']].first().reset_index(), on='BS', how='left')

df=data.copy()

In [ ]:
df

,Time,BS,Energy,CellName,load,ESMode1,ESMode2,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,1/1/2023 1:00,B_0,64.275037,Cell0,0.487936,0.0,0.0,0.000000,Type1,Mode2,365.00,20,4,6.875934
1,1/1/2023 2:00,B_0,55.904335,Cell0,0.344468,0.0,0.0,0.000000,Type1,Mode2,365.00,20,4,6.875934
2,1/1/2023 3:00,B_0,57.698057,Cell0,0.193766,0.0,0.0,0.000000,Type1,Mode2,365.00,20,4,6.875934
3,1/1/2023 4:00,B_0,55.156951,Cell0,0.222383,0.0,0.0,0.000000,Type1,Mode2,365.00,20,4,6.875934
4,1/1/2023 5:00,B_0,56.053812,Cell0,0.175436,0.0,0.0,0.000000,Type1,Mode2,365.00,20,4,6.875934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98079,1/2/2023 17:00,B_1018,14.648729,Cell0,0.087538,0.0,0.0,0.781566,Type5,Mode2,426.98,2,2,7.325859
98080,1/2/2023 18:00,B_1018,14.648729,Cell0,0.082635,0.0,0.0,0.804310,Type5,Mode2,426.98,2,2,7.325859
98081,1/2/2023 21:00,B_1018,13.452915,Cell0,0.055538,0.0,0.0,0.846935,Type5,Mode2,426.98,2,2,7.325859
98082,1/2/2023 22:00,B_1018,13.602392,Cell0,0.058077,0.0,0.0,0.856434,Type5,Mode2,426.98,2,2,7.325859


In [ ]:
df.columns

Index(['Time', 'BS', 'Energy', 'CellName', 'load', 'ESMode1', 'ESMode2',
       'ESMode6', 'RUType', 'Mode', 'Frequency', 'Bandwidth', 'Antennas',
       'TXpower'],
      dtype='object')

In [ ]:
df.describe()

,Energy,load,ESMode1,ESMode2,ESMode6,Frequency,Bandwidth,Antennas,TXpower
count,98084.000000,98084.000000,98084.000000,98084.000000,98084.000000,98084.000000,98084.000000,98084.000000,98084.000000
mean,28.986888,0.242025,0.039012,0.037728,0.030367,368.979381,15.965917,2.186473,6.733521
std,14.345648,0.236798,0.187290,0.184540,0.149012,136.286959,5.645188,2.033932,0.304979
min,0.747384,0.000000,0.000000,0.000000,0.000000,155.600000,2.000000,1.000000,5.381166
25%,18.535127,0.055922,0.000000,0.000000,0.000000,189.000000,10.000000,2.000000,6.427504
50%,24.962631,0.157570,0.000000,0.000000,0.000000,365.000000,20.000000,2.000000,6.875934
75%,37.668161,0.358069,0.000000,0.000000,0.000000,532.000000,20.000000,2.000000,6.875934
max,100.000000,0.993957,1.000000,1.000000,0.931032,979.998000,20.000000,64.000000,8.375336


In [ ]:
# df = df.merge(energy, on=['Time', 'BS'], how='left')
# df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98084 entries, 0 to 98083
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Time       98084 non-null  object 
 1   BS         98084 non-null  object 
 2   Energy     98084 non-null  float64
 3   CellName   98084 non-null  object 
 4   load       98084 non-null  float64
 5   ESMode1    98084 non-null  float64
 6   ESMode2    98084 non-null  float64
 7   ESMode6    98084 non-null  float64
 8   RUType     98084 non-null  object 
 9   Mode       98084 non-null  object 
 10  Frequency  98084 non-null  float64
 11  Bandwidth  98084 non-null  int64  
 12  Antennas   98084 non-null  int64  
 13  TXpower    98084 non-null  float64
dtypes: float64(7), int64(2), object(5)
memory usage: 11.2+ MB


In [ ]:
!pip install mlflow

In [ ]:
MLFLOW_TRACKING_URI="https://dagshub.com/samithkavishke/5g-energy-consumption-modeling.mlflow"
MLFLOW_TRACKING_USERNAME="Kajaani-Balabavan"
MLFLOW_TRACKING_PASSWORD='76352022f25ef66bfc854478c2caf765d3ab72c4'

In [ ]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("Competition")

<Experiment: artifact_location='mlflow-artifacts:/05e4fbc588934e979ba8497f14f54669', creation_time=1698146260875, experiment_id='0', last_update_time=1698146260875, lifecycle_stage='active', name='Competition', tags={}>

In [ ]:
os.environ['MLFLOW_TRACKING_USERNAME'] = MLFLOW_TRACKING_USERNAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = MLFLOW_TRACKING_PASSWORD

In [ ]:
df['BS'] = df['BS'].str.replace('B_', '').astype(int)
# df['RUType'] = df['RUType'].str.replace('Type', '').astype(int)
# df['Mode'] = df['Mode'].str.replace('Mode', '').astype(int)
df['Time'] = pd.to_datetime(df['Time'])
df['hour'] = df['Time'].dt.hour
df = pd.get_dummies(df, columns=['RUType',  'Mode', 'hour'])
df.drop(['Time'], axis=1, inplace=True)
df.head()

,BS,Energy,CellName,load,ESMode1,ESMode2,ESMode6,Frequency,Bandwidth,Antennas,TXpower,RUType_Type1,RUType_Type10,RUType_Type11,RUType_Type12,RUType_Type2,RUType_Type3,RUType_Type4,RUType_Type5,RUType_Type6,RUType_Type7,RUType_Type8,RUType_Type9,Mode_Mode1,Mode_Mode2,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,0,64.275037,Cell0,0.487936,0.0,0.0,0.0,365.0,20,4,6.875934,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,55.904335,Cell0,0.344468,0.0,0.0,0.0,365.0,20,4,6.875934,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,57.698057,Cell0,0.193766,0.0,0.0,0.0,365.0,20,4,6.875934,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,55.156951,Cell0,0.222383,0.0,0.0,0.0,365.0,20,4,6.875934,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,56.053812,Cell0,0.175436,0.0,0.0,0.0,365.0,20,4,6.875934,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df['CellName'] = df['CellName'].str.replace('Cell', '').astype(int)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer  # Import the imputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow

# Define your features (X) and target variable (y)
X = df.drop("Energy", axis=1)  # Features (excluding the "Energy" column)
y = df["Energy"]  # Target variable

# Initialize the imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)  # Apply imputer to X

# Define your Linear Regression model
model = LinearRegression()

# Create a KFold cross-validation object with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store the scores at each fold
mae_scores = []
rmse_scores = []
r2_scores = []

# Perform cross-validation and get the scores at each fold
for i, (train_idx, valid_idx) in enumerate(kf.split(X_imputed)):
    X_train_fold, X_valid_fold = X_imputed[train_idx], X_imputed[valid_idx]
    y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

    # Fit the model
    model.fit(X_train_fold, y_train_fold)

    # Make predictions
    y_pred = model.predict(X_valid_fold)

    # Calculate evaluation metrics
    mae = mean_absolute_error(y_valid_fold, y_pred)
    rmse = np.sqrt(mean_squared_error(y_valid_fold, y_pred))
    r2 = r2_score(y_valid_fold, y_pred)

    # Append scores to the lists
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

    # Print and log the results for each fold
    print(f"Fold {i + 1}:")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R-squared: {r2}")

# Calculate the means of the scores
mean_mae = np.mean(mae_scores)
mean_rmse = np.mean(rmse_scores)
mean_r2 = np.mean(r2_scores)

# Print and log the overall results
print("=*" * 50)
print("Overall Results:")
print(f"Mean Absolute Error (MAE): {mean_mae}")
print(f"Root Mean Squared Error (RMSE): {mean_rmse}")
print(f"R-squared (R2) Score: {mean_r2}")

# Logging parameters and metrics using MLflow
with mlflow.start_run():
    mlflow.log_metric("MAE", mean_mae)
    mlflow.log_metric("RMSE", mean_rmse)
    mlflow.log_metric("R2", mean_r2)

    # Log the imputer as an artifact
    mlflow.sklearn.log_model(imputer, "imputer")

    # Log the trained model as an artifact
    mlflow.sklearn.log_model(model, "linear_regression_model")

# End the MLflow run
mlflow.end_run()

Fold 1:
MAE: 3.61753792224496
RMSE: 4.896092629555733
R-squared: 0.8833545572669316
Fold 2:
MAE: 3.606712659251027
RMSE: 4.908665751621745
R-squared: 0.8840988923848178
Fold 3:
MAE: 3.640304514679034
RMSE: 4.966568288858854
R-squared: 0.8771792186514269
Fold 4:
MAE: 3.58882686339979
RMSE: 4.909293606748236
R-squared: 0.8832496995622495
Fold 5:
MAE: 3.6255741203878173
RMSE: 5.303629615863752
R-squared: 0.8649391317446238
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
Overall Results:
Mean Absolute Error (MAE): 3.615791215992526
Root Mean Squared Error (RMSE): 4.996849978529664
R-squared (R2) Score: 0.8785642999220098


2023/11/01 16:48:29 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import KFold
# from sklearn.linear_model import LinearRegression
# from sklearn.impute import SimpleImputer  # Import the imputer
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import mlflow

# # Define your features (X) and target variable (y)
# X = df.drop("Energy", axis=1)  # Features (excluding the "Energy" column)
# y = df["Energy"]  # Target variable

# # Initialize the imputer to fill missing values with the mean
# imputer = SimpleImputer(strategy='mean')
# X = imputer.fit_transform(X)  # Apply imputer to X

# # Define your Linear Regression model
# model = LinearRegression()

# # Create a KFold cross-validation object with 5 splits
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Initialize lists to store the scores at each fold
# mae_scores = []
# rmse_scores = []
# r2_scores = []

# # Perform cross-validation and get the scores at each fold
# for i, (train_idx, valid_idx) in enumerate(kf.split(X)):
#     X_train_fold, X_valid_fold = X[train_idx], X[valid_idx]
#     y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

#     # Fit the model
#     model.fit(X_train_fold, y_train_fold)

#     # Make predictions
#     y_pred = model.predict(X_valid_fold)

#     # Calculate evaluation metrics
#     mae = mean_absolute_error(y_valid_fold, y_pred)
#     rmse = np.sqrt(mean_squared_error(y_valid_fold, y_pred))
#     r2 = r2_score(y_valid_fold, y_pred)

#     # Append scores to the lists
#     mae_scores.append(mae)
#     rmse_scores.append(rmse)
#     r2_scores.append(r2)

#     # Print and log the results for each fold
#     print(f"Fold {i + 1}:")
#     print(f"MAE: {mae}")
#     print(f"RMSE: {rmse}")
#     print(f"R-squared: {r2}")

# # Calculate the means of the scores
# mean_mae = np.mean(mae_scores)
# mean_rmse = np.mean(rmse_scores)
# mean_r2 = np.mean(r2_scores)

# # Print and log the overall results
# print("=*" * 50)
# print("Overall Results:")
# print(f"Mean Absolute Error (MAE): {mean_mae}")
# print(f"Root Mean Squared Error (RMSE): {mean_rmse}")
# print(f"R-squared (R2) Score: {mean_r2}")

# # Logging parameters and metrics using MLflow
# with mlflow.start_run():
#     mlflow.log_metric("MAE", mean_mae)
#     mlflow.log_metric("RMSE", mean_rmse)
#     mlflow.log_metric("R2", mean_r2)
